# Przetworzenie obrazów przez GroundingSAM

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys

In [2]:
sys.path.append(str(Path().resolve() / "Grounded-SAM-2"))

from torchvision.ops import box_convert
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
from grounding_dino.groundingdino.util.inference import load_model, load_image, predict
import torch

2025-06-17 11:45:36.543986: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-17 11:45:36.544044: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-17 11:45:36.545139: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-17 11:45:36.552489: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-17 11:45:37.511901: W tensorflow/compiler/tf2

Ustawienie opcji wyświetlania w pandas (opcjonalne)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [4]:
SAVE_DIR = Path("outputs")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

## Pobranie danych

Pobranie ścieżek do plików w odpowiedniej kolejności. Następnie połączenie z danymi w pliku .csv tak, aby sobie odpowidały.

In [5]:
DATA_PATH = Path("img")

# Załoadowanie CSV
df_1 = pd.read_csv(DATA_PATH / "icm_111/imitation_control_model_2025_03_25-14_39_50.csv", sep=" ")
df_2 = pd.read_csv(DATA_PATH / "icm_61/imitation_control_model_2024_12_03-12_31_55.csv", sep=" ")

# Folder z obrazami
image_folder_1 = Path(DATA_PATH / "icm_111")
image_folder_2 = Path(DATA_PATH / "icm_61")

# Funkcja zwracająca ścieżkę do obrazu kolorowego
def get_color_path_1(row):
    return image_folder_1 / f"imitation_control_model_2025_03_25-14_39_50_{int(row['id'])}_color_orig.png"
def get_color_path_2(row):
    return image_folder_2 / f"imitation_control_model_2024_12_03-12_31_55_{int(row['id'])}_color_orig.png"

# Funkcja zwracająca ścieżkę do obrazu głębi
def get_depth_path_1(row):
    return image_folder_1 / f"imitation_control_model_2025_03_25-14_39_50_{int(row['id'])}_depth.png"
def get_depth_path_2(row):
    return image_folder_2 / f"imitation_control_model_2024_12_03-12_31_55_{int(row['id'])}_depth.png"

# Dodanie kolumn z ścieżkami
df_1['color_path'] = df_1.apply(get_color_path_1, axis=1)
df_1['depth_path'] = df_1.apply(get_depth_path_1, axis=1)

# Dodanie kolumn z ścieżkami
df_2['color_path'] = df_2.apply(get_color_path_2, axis=1)
df_2['depth_path'] = df_2.apply(get_depth_path_2, axis=1)

# Połączenie DataFrameów
df = pd.concat([df_1, df_2], ignore_index=True)

# Wyświetlenie pierwszych 12 wierszy
df.head()

,id,time,ESJoint1,ESJoint2,ESJoint3,ESJoint4,ESJoint5,ESJoint6,gripper_finger_1_joint,gripper_finger_2_joint,color_path,depth_path
0,0,1.742910e+09,3.848058,0.729946,1.842714,1.702291,0.986970,3.103531,0.603365,0.899934,img/icm_111/imitation_control_model_2025_03_25-14_39_50_0_color_orig.png,img/icm_111/imitation_control_model_2025_03_25-14_39_50_0_depth.png
1,1,1.742910e+09,3.847902,0.730181,1.842891,1.702339,0.986970,3.103531,0.603365,0.899934,img/icm_111/imitation_control_model_2025_03_25-14_39_50_1_color_orig.png,img/icm_111/imitation_control_model_2025_03_25-14_39_50_1_depth.png
2,2,1.742910e+09,3.847784,0.730299,1.842949,1.702291,0.987018,3.103483,0.603365,0.899934,img/icm_111/imitation_control_model_2025_03_25-14_39_50_2_color_orig.png,img/icm_111/imitation_control_model_2025_03_25-14_39_50_2_depth.png
3,3,1.742910e+09,3.847725,0.730377,1.842969,1.702291,0.987018,3.103339,0.603365,0.899934,img/icm_111/imitation_control_model_2025_03_25-14_39_50_3_color_orig.png,img/icm_111/imitation_control_model_2025_03_25-14_39_50_3_depth.png
4,4,1.742910e+09,3.847686,0.730416,1.842989,1.702291,0.987114,3.103291,0.603365,0.899934,img/icm_111/imitation_control_model_2025_03_25-14_39_50_4_color_orig.png,img/icm_111/imitation_control_model_2025_03_25-14_39_50_4_depth.png


In [6]:
df.tail()

,id,time,ESJoint1,ESJoint2,ESJoint3,ESJoint4,ESJoint5,ESJoint6,gripper_finger_1_joint,gripper_finger_2_joint,color_path,depth_path
167,56,1.733226e+09,3.083501,1.766065,2.012979,0.917516,1.592701,2.319139,0.194304,1.360128,img/icm_61/imitation_control_model_2024_12_03-12_31_55_56_color_orig.png,img/icm_61/imitation_control_model_2024_12_03-12_31_55_56_depth.png
168,57,1.733226e+09,3.111521,1.591545,2.156014,0.948148,1.593036,2.346703,0.777216,0.726083,img/icm_61/imitation_control_model_2024_12_03-12_31_55_57_color_orig.png,img/icm_61/imitation_control_model_2024_12_03-12_31_55_57_depth.png
169,58,1.733226e+09,3.363226,1.831601,1.642108,1.217649,1.589297,2.598947,0.194304,1.360128,img/icm_61/imitation_control_model_2024_12_03-12_31_55_58_color_orig.png,img/icm_61/imitation_control_model_2024_12_03-12_31_55_58_depth.png
170,59,1.733226e+09,3.385900,1.917128,1.739089,1.035057,1.588866,2.621909,0.777216,0.720970,img/icm_61/imitation_control_model_2024_12_03-12_31_55_59_color_orig.png,img/icm_61/imitation_control_model_2024_12_03-12_31_55_59_depth.png
171,60,1.733226e+09,3.385900,1.917128,1.739089,1.035057,1.588866,2.621909,0.777216,0.720970,img/icm_61/imitation_control_model_2024_12_03-12_31_55_60_color_orig.png,img/icm_61/imitation_control_model_2024_12_03-12_31_55_60_depth.png


Zapisanie dataframe do pliku .csv

In [7]:
df.to_csv(str(SAVE_DIR)+'/df.csv', index=False)

# Konfiguracja GDSAM

In [8]:
# Wczytanie do stałych ścieżek do modeli i konfiguracji
SAM2_CONFIG = "configs/sam2.1/sam2.1_hiera_l.yaml"
SAM2_CHECKPOINT = "Grounded-SAM-2/checkpoints/sam2.1_hiera_large.pt"
GROUNDING_DINO_CONFIG = "Grounded-SAM-2/grounding_dino/groundingdino/config/GroundingDINO_SwinB_cfg.py"
GROUNDING_DINO_CHECKPOINT = "Grounded-SAM-2/gdino_checkpoints/groundingdino_swinb_cogcoor.pth"

# Ustawienie odpowiednich parametrów
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if torch.cuda.get_device_properties(0).major >= 8:
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

In [9]:
# załadaowanie modelu SAM
sam2_model = build_sam2(SAM2_CONFIG, SAM2_CHECKPOINT, device=DEVICE)
sam2_predictor = SAM2ImagePredictor(sam2_model)

# załadaowanie modelu GD
grounding_model = load_model(
    model_config_path=GROUNDING_DINO_CONFIG, 
    model_checkpoint_path=GROUNDING_DINO_CHECKPOINT,
    device=DEVICE
)

final text_encoder_type: bert-base-uncased


# Detekcja i segmentacja

In [10]:
incomplete_detect = []
masks_array = []

# Tekstowe prompty dla detekcji
TEXT_PROMPTS = ["pink robot finger gripper on the robot gripper.", "green small rectangle on the robot gripper.", "blue robot finger gripper on the robot gripper."]

# Próg detekcji dla boxów i tekstu
BOX_TRESHOLD = 0.28
TEXT_TRESHOLD = 0.3

# Parametr odfiltrowania dużych masek
AREA_THRESHOLD = 2300

for count_iter, i in enumerate(range(0, len(df))):

  # Załadowanie obrazu kolorowego
  image_source, image = load_image(df['color_path'][i])
  h, w, _ = image_source.shape
  sam2_predictor.set_image(image_source)

  # Załadowanie obrazu głębokości
  depth_img, _ = load_image(df['depth_path'][i])
  depth_img = depth_img[:, :, 0]
  
  # Tymczasowa tablica do przechowywania masek
  tmp = np.zeros((len(TEXT_PROMPTS), h, w), dtype=np.float32)

  # Iteracja po trzech obiektach
  for j in range(len(TEXT_PROMPTS)):

    # Wybór tekstu dla obiektu
    text_prompt = TEXT_PROMPTS[j]

    # Detekcja obiektów
    boxes, confidences, labels = predict(
        model=grounding_model,
        image=image,
        caption=text_prompt,
        box_threshold=BOX_TRESHOLD,
        text_threshold=TEXT_TRESHOLD)

    # Konwersja boxów z cxcywh na xyxy
    boxes2 = boxes * torch.Tensor([w, h, w, h])
    xyxy = box_convert(boxes=boxes2, in_fmt="cxcywh", out_fmt="xyxy").numpy()

    # Sprawdzenie czy są detekcje
    print(f"Obraz {i}, Obiekt {j}: Liczba wykrytych masek: {len(xyxy)}")
    print(f"xyxy = {xyxy}")
    if xyxy.size == 0:
        print(f"Brak detekcji dla '{text_prompt}' na obrazie {i}")
        incomplete_detect.append(i)
        #tmp.extend([0, 0, 0,])
        continue
    
    # Segmentacja obiektów
    masks, scores, logits = sam2_predictor.predict(
        point_coords=None,
        point_labels=None,
        box=xyxy,
        multimask_output=False,
    )

    """
    Pzetwarzanie po detekcji i segmentacji
    """

    # Zamiana wymiarów do (n, H, W)
    if masks.ndim == 4:
        masks = masks.squeeze(1)

    # Obliczenie pola segmentacji
    mask_areas = np.sum(masks, axis=(1, 2))  # Obliczenie pola segmentacji
    valid_mask_indices = np.where(mask_areas < AREA_THRESHOLD)[0] # Zachowanie segmentacji poniżej 2300 pikseli

    # Sprawdzenie czy są maski
    if len(valid_mask_indices) == 0:
        print(f"Obraz {i}, Obiekt {j}: Brak maski z polem < {AREA_THRESHOLD} pikseli")
        #tmp.extend([0, 0, 0,])
        incomplete_detect.append(i)
        continue 

    # Wybór maski o największym współczynniku pewności (confidence)
    confidences = confidences.numpy()
    valid_confidences = confidences[valid_mask_indices]
    best_mask_idx = valid_mask_indices[np.argmax(valid_confidences)]

    # Wybór maski o największym współczynniku pewności (confidence)
    selected_mask = masks[best_mask_idx:best_mask_idx+1]  # Zachowanie wymiarów (1, H, W)
    selected_xyxy = xyxy[best_mask_idx:best_mask_idx+1]  # Zachowanie wymiarów (1, 4)
    selected_confidence = confidences[best_mask_idx]
    selected_label = labels[best_mask_idx]
    selected_area = mask_areas[best_mask_idx]

    # Wyświetlenie informacji
    print(f"Numer obrazu: {i}")
    print(f"Tekst wejściowy: {text_prompt}")
    print(f"Maska - Pole: {selected_area} pikseli, Pewność: {selected_confidence:.2f}")

    # Zapisanie maski do tablicy
    mask = selected_mask[0]
    tmp[j] = mask

    """
    Wizualizacja i wyświetlanie
    """

    # # Zmienne do wizualizacji
    # class_ids = np.array([0]) 
    # labels = [f"{selected_label} {selected_confidence:.2f}"]

    # # Detekcje do wizuazlizacji
    # detections = sv.Detections(
    #     xyxy=selected_xyxy,  # (1, 4)
    #     mask=selected_mask.astype(bool),  # (1, H, W)
    #     class_id=class_ids
    # )

    # # Wizualizacja prostokątów
    # box_annotator = sv.BoxAnnotator()
    # annotated_frame = box_annotator.annotate(scene=cv2.cvtColor(image_source, cv2.COLOR_BGR2RGB), detections=detections)

    # # Wizualizacja etykiet
    # label_annotator = sv.LabelAnnotator()
    # annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)

    # # Wizualizacja masek
    # mask_annotator = sv.MaskAnnotator()
    # annotated_frame = mask_annotator.annotate(scene=annotated_frame, detections=detections)
    # sv.plot_image(annotated_frame)

  # Zapisanie masek do tablicy
  masks_array.append(tmp)

masks_array = np.array(masks_array)
print(f"masks_array -> {masks_array.shape}")

# Niekompletne detekcje
incomplete_detect = np.array(list(set(incomplete_detect)))
incomplete_detect = np.unique(incomplete_detect)
print(f"Liczba niekompletnych id: {incomplete_detect.size}. Liczba iteracji: {count_iter+1}. Procent odrzuconych: {incomplete_detect.size/(count_iter+1)*100}%")
print(f"Lista niekompletnych id: {incomplete_detect}")

Obraz 0, Obiekt 0: Liczba wykrytych masek: 3
xyxy = [[286.7528   115.42283  304.34058  132.3482  ]
 [279.27148  113.48024  312.8872   140.51772 ]
 [  0.590683 135.08127  510.72723  381.16898 ]]
Numer obrazu: 0
Tekst wejściowy: pink robot finger gripper on the robot gripper.
Maska - Pole: 458.0 pikseli, Pewność: 0.38


Falling back to all available kernels for scaled_dot_product_attention (which may have a slower speed).


Obraz 0, Obiekt 1: Liczba wykrytych masek: 1
xyxy = [[  0.516037 134.80237  510.43835  380.42426 ]]
Obraz 0, Obiekt 1: Brak maski z polem < 2300 pikseli
Obraz 0, Obiekt 2: Liczba wykrytych masek: 1
xyxy = [[279.24176 114.62065 315.1311  141.20401]]
Numer obrazu: 0
Tekst wejściowy: blue robot finger gripper on the robot gripper.
Maska - Pole: 461.0 pikseli, Pewność: 0.31
Obraz 1, Obiekt 0: Liczba wykrytych masek: 3
xyxy = [[285.79965   115.642494  304.7101    133.08139  ]
 [279.659     113.917404  313.44415   140.5655   ]
 [  1.1150818 135.08798   510.29355   381.20523  ]]
Numer obrazu: 1
Tekst wejściowy: pink robot finger gripper on the robot gripper.
Maska - Pole: 450.0 pikseli, Pewność: 0.39
Obraz 1, Obiekt 1: Liczba wykrytych masek: 1
xyxy = [[  0.81095886 134.9701     510.20306    380.55835   ]]
Obraz 1, Obiekt 1: Brak maski z polem < 2300 pikseli
Obraz 1, Obiekt 2: Liczba wykrytych masek: 2
xyxy = [[135.89044    8.131996 314.0759   142.35754 ]
 [279.9291   114.90787  315.067    14

Zapisanie tablic z danymi do pliku

In [11]:
np.save(SAVE_DIR / 'masks_array.npy', masks_array)
np.save(SAVE_DIR / 'incomplete_detect.npy', incomplete_detect)